In [1]:
import pandas as pd
from docplex.mp.model import Model

#============================================================================================================================================
# Retirando dados das planilhas
#============================================================================================================================================

    
read_capacidade = pd.read_csv("DadosBergamini2022/Instancia I/capacidade.csv")
read_demanda = pd.read_csv("DadosBergamini2022/Instancia I/demanda.csv")
read_itens = pd.read_csv("DadosBergamini2022/Instancia I/modelos.csv")
read_mapa_modelo_linha = pd.read_csv("DadosBergamini2022/Instancia I/mapalinhamodelo.csv")
read_mapa_linha_recurso = pd.read_csv("DadosBergamini2022/Instancia I/mapalinharecurso.csv")
read_mapa_modelo_recurso = pd.read_csv("DadosBergamini2022/Instancia I/mapamodelorecurso.csv")
read_C = pd.read_csv("DadosBergamini2022/Instancia I/C.csv")
read_t_processamento = pd.read_csv("DadosBergamini2022/Instancia I/tprocessamento.csv")
read_capacidade_diaria = pd.read_csv("DadosBergamini2022/Instancia I/capacidadediaria.csv")

In [2]:
#============================================================================================================================================
    
# Lendo modelos e criando um dicionário entre o nome e o número dos modelos

#============================================================================================================================================


itens_list = read_itens['modelos'].tolist()
    
lista_aux = []
for i in range(1, len(itens_list)+1):
    lista_aux.append(i)
    
dict_aux = {}
for keys in itens_list:
    for values in lista_aux:
        dict_aux[keys] = values
        lista_aux.remove(values)
        break

#============================================================================================================================================
    
# Lendo a capacidade de linhas por período

#============================================================================================================================================

C_list_c = read_C['C'].to_list()
C_list_linhas = read_C['linhas'].to_list()
C_list_periodos = read_C['periodos'].to_list()

num_linhas = max(C_list_linhas) + 1
num_periodos = max(C_list_periodos) + 1

matriz_lin_per = [[0] * num_periodos for i in range(num_linhas)]

for c, linha, periodo in zip(C_list_c, C_list_linhas, C_list_periodos):
    matriz_lin_per[linha][periodo] = c


#============================================================================================================================================
    
# Lendo o tempo de processamento de cada modelo em cada linha

#============================================================================================================================================


list_modelos = read_t_processamento['modelos'].to_list()
list_linhas = read_t_processamento['linhas'].to_list()
list_temp = read_t_processamento['tprocessamento'].to_list()

list_modelos_num = []
for i in list_modelos:
    list_modelos_num.append(dict_aux[i])

matriz_tempo = {}
for modelo, linha, t in zip(list_modelos_num, list_linhas, list_temp):
    if modelo not in matriz_tempo:
        matriz_tempo[modelo] = {}
    matriz_tempo[modelo][linha] = t



#============================================================================================================================================
    
    # Declarando os conjuntos
    #============================================================================================================================================


    CONJ_P_itens = list(range(1,len(itens_list)+1))
    CONJ_R_rec = read_mapa_modelo_recurso['recursos'].tolist()
    CONJ_T_per = list(dict.fromkeys(C_list_periodos))
    CONJ_L_lin = list(dict.fromkeys(C_list_linhas))

#============================================================================================================================================
    
    # Declarando os subconjuntos
    #============================================================================================================================================

	#MAPA MODELO LINHA

    linhas_MLM = read_mapa_modelo_linha['linhas'].tolist()
    modelos_MLM = read_mapa_modelo_linha['modelos'].tolist()

    modelos_valor_numerico = []

    for i in modelos_MLM:
        modelos_valor_numerico.append(dict_aux[i])


    SUBCONJ_modelo_linha = []

    for i in itens_list:
        aux = []
        SUBCONJ_modelo_linha.append(aux)

    contador = 0

    for i in modelos_valor_numerico:
        SUBCONJ_modelo_linha[i-1].append(linhas_MLM[contador])
        contador += 1

	#MAPA MODELO RECURSO
	
    recursos_MMR = read_mapa_modelo_recurso['recursos'].tolist() 
    modelos_MMR = read_mapa_modelo_recurso['modelos'].tolist()

    modelos_MMR_numerico = []

    for i in modelos_MMR:
        modelos_MMR_numerico.append(dict_aux[i])

    SUBCONJ_recurso_modelo = []

    for i in range(max(recursos_MMR)):
        aux = []
        SUBCONJ_recurso_modelo.append(aux)

    contador = 0

    for r in recursos_MMR:
        SUBCONJ_recurso_modelo[r-1].append(modelos_MMR_numerico[contador])
        contador += 1

In [3]:
#============================================================================================================================================
    
# Declarando os limites de cada conjunto

#============================================================================================================================================




P = len(CONJ_P_itens)
R = len(CONJ_R_rec)
T = len(CONJ_T_per)
L = len(CONJ_L_lin)


#============================================================================================================================================
  
# Declarando as capacidades

#============================================================================================================================================


Cr = read_capacidade['capacidade'].tolist()
Cd = read_capacidade_diaria['capacidadediaria'].tolist()
Ch = matriz_lin_per
p = matriz_tempo

#============================================================================================================================================
  
# Declarando as demandas

#============================================================================================================================================
    
Q = 240   #fixo
q = 20    #fixo
d = read_demanda['demanda'].to_list()
d_num_lotes = d
d_tam_residual = d
   
H = 1000
    
#print("Itens ", CONJ_P_itens, "Quantidade: ", P, "\n")
#print("Recursos ", CONJ_R_rec, "Quantidade: ", R, "\n")
#print("Períodos ", CONJ_T_per, "Quantidade: ", T, "\n")
#print("Linhas", CONJ_L_lin, "Quantidade: ",  L, "\n")
#print("Demanda", d, "Quantidade: ", len(d), "\n")
#print("Capacidade por recurso", Cr, "Quantidade: ", len(Cr), "\n")
#print("Capacidade diaria do recurso", Cd, "Quantidade: ", len(Cd), "\n")
#print("Capacidade da linha em cada Período", Ch, "Quantidade: ", len(Ch), "por", len(Ch[0]), "\n")
#print("Tempo de processamento para cada Modelo em cada linha", p, '\n')

#============================================================================================================================================


In [4]:
#============================================================================================================================================
    
    # Criando modelo
#============================================================================================================================================

model = Model(name = "Modelo DLMPF")

#============================================================================================================================================
    
# Declarando as variáveis de decisão

#============================================================================================================================================


x = {}
y = {}
z = {}
w = {}
    
x = model.integer_var_cube(keys1=range(1,P+1), keys2=range(1,L+1), keys3=range(0,T+1), name='X')
y = model.binary_var_cube(keys1=range(1,P+1), keys2=range(1,L+1), keys3=range(0,T+1), name='Y')
w = model.integer_var_cube(keys1=range(1,P+1), keys2=range(1,L+1), keys3=range(0,T+1), name='W')
z = model.binary_var_cube(keys1=range(1,P+1), keys2=range(1,L+1), keys3=range(0,T+1), name='Z')
    



In [ ]:
#============================================================================================================================================
   
# Declarando as restrições

#============================================================================================================================================


# Restrição 2
for i in CONJ_P_itens:
    model.add_constraint(model.sum(Q * x[i,k,t] + d_num_lotes[i-1] * y[i,k,t] for k in CONJ_L_lin for t in CONJ_T_per) == d[i-1], ctname = 'Restrição 2')
    
# Restrição 3
    
for i in CONJ_P_itens:
    model.add_constraint(model.sum(x[i,k,t] for k in range(1,L+1) for t in range(T+1)) == d_tam_residual[i-1], ctname = 'Restrição 3')
        
# Restrição 4
    
for i in CONJ_P_itens:
    model.add_constraint(model.sum(y[i, k, t] for k in range(1,L+1) for t in range(T+1)) <= 1, ctname = 'Restrição 4')
    
# Restrição 5
    
for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        for t in CONJ_T_per:
            model.add_constraint(w[i, k, t] <= Q * x[i, k, t] + d_num_lotes[i-1] * y[i, k, t], ctname = 'Restrição 5')
    
# Restrição 6
    
for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        for t in CONJ_T_per:
            model.add_constraint(w[i, k, t] <= (Q - q) * z[i,k,t], ctname = 'Restrição 6')
                
# Restrição 7
    
for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        for t in CONJ_T_per:
            model.add_constraint(q * z[i,k,t] <= w[i,k,t], ctname = 'Restrição 7')
                
# Restrição 8
for k in CONJ_L_lin:
        for t in CONJ_T_per:
            model.add_constraint(model.sum(z[i,k,t] for i in CONJ_P_itens)<=1, ctname = 'Restrição 8')
                
# Restrição 9
    
for r in CONJ_R_rec:
    model.add_constraint(model.sum(Q * x[i,k,t] + d_num_lotes[i-1] * y[i,k,t] for i in range(1,P+1) for k in range(1,L+1) for t in range(T+1)) <= Cr[r-1], ctname = "Restrição 9")
    
# Restrição 10
    
for r in CONJ_R_rec:
    for t in CONJ_T_per:
        if t != 0:
            model.add_constraint(model.sum(Q * x[i,k,t] + d_num_lotes[i-1] * y[i,k,t] + w[i, k, t-1] - w[i,k,t] for i in range (1,P+1) for k in range(1,L+1)) <= Cd[r-1], ctname = "Restrição 10")
        else:
            model.add_constraint(model.sum(Q * x[i,k,t] + d_num_lotes[i-1] * y[i,k,t] + 0 - w[i,k,t] for i in range (1,P+1) for k in range(1,L+1)) <= Cd[r-1], ctname = "Restrição 10")
    
# Restrição 11
    for k in CONJ_L_lin:
        for t in CONJ_T_per:
            if t != 0:
                model.add_constraint(model.sum(p[i][k] * (Q * x[i,k,t] + (d[i-1] % Q) * y[i,k,t] + w[i,k,t-1] - w[i,k,t]) for i in CONJ_P_itens if i in p and k in p[i]) <= Ch[k-1][t-1], ctname='Restrição 11')
    
# Restrição 12
    
for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        model.add_constraint(w[i,k,0] == 0, ctname = 'Restrição 12')
            
# Restrição 13
            
for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        model.add_constraint(w[i,k,T] == 0, ctname = 'Restrição 13')
    
            
#Restrição 14
for i in CONJ_P_itens:
    for k in CONJ_L_lin:
        for t in CONJ_T_per:
            model.add_constraint(0<=w[i,k,t], ctname = 'Restrição 14.1')
            model.add_constraint(w[i,k,t]<= Q - q, ctname = 'Restrição 14.2')
    



In [ ]:
#============================================================================================================================================
  
# Declarando função objetivo

#============================================================================================================================================

                
objetivo = model.sum(
        p[i][k] * Q * x[i, k, t] + p[i][k] * (d[i-1] % Q) * y[i, k, t] + z[i, k, t] 
        for i in CONJ_P_itens 
        for k in CONJ_L_lin 
        for t in CONJ_T_per 
        if t > 0 and i in p and k in p[i])

model.minimize(objetivo)



In [ ]:
#============================================================================================================================================
   
# Exportando dados do modelo

#============================================================================================================================================


print(model.export_as_lp_string())

In [ ]:
#============================================================================================================================================
   
# Resolução do modelo

#============================================================================================================================================

#sol = model.solve()
#print(sol)



In [ ]:
#============================================================================================================================================
   
# Valores de cada variável

#============================================================================================================================================

#for var in m1.iter_variables():
#    print(f'Variável {var} = {var.solution_value} ({var.vartype})')